### Matching Data to Datastream Historical Stocks prices, MV, and Volumes

By Xiaoran (Jason) Jia, Oct 2022

- Step 1: Establish matchable firms -- Using SEDOL code and CUSIP code，create seperate lists firms that are matchable to Datastream-readable codes 
- Step 2: Query the lists (performed in Excel) - long time and tedious task.
- Step 3: match the queried lists to the sample - The merging process is extremly time-consuming (unless the machine is superior in performance and RAM)
- Step 4: match the data to benchmark indices returns
- Step 5: Perform calculations to get the three categories of measurements

In [3]:
# Import all modules required
import pandas as pd
import numpy as np
import re
import datetime as dt
from datetime import datetime, date
from pandas.tseries.offsets import *
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 20)

### Step 1: Establish Matchable Firms

#### 1.1 Import and modify sample file

New variables created are, for example:

- 'ann_p0' means earnings announcement date;
- 'ann_p1' means earnings announcement date plus one business day;
- 'ann_m1' means earnings announcement date minus one business day;
- 'ann_m120' means earnings announcement date minus 120 business days;

In [42]:
# Import
ann = pd.read_sas('anndate.sas7bdat', format='sas7bdat', encoding="utf-8", )

# Modify column types and names
ann['CONM'] = ann['CONM'].astype(str)
ann['SEDOL'] = ann['SEDOL'].astype(str)
ann['GVKEY'] = ann['GVKEY'].astype(str)
ann['CUSIP'] = ann['CUSIP'].astype(str)
ann.rename(columns={'ANNDATS_ACT':'ann_p0'}, inplace=True)

# Add +1, +2 and -1, -2 columns
ann['ann_p1'] = ann['ann_p0'] + BusinessDay()
ann['ann_p2'] = ann['ann_p0'] + 2*BusinessDay()
ann['ann_m1'] = ann['ann_p0'] - BusinessDay()
ann['ann_m2'] = ann['ann_p0'] - 2*BusinessDay()

# Add -120 to -21 columns
for i in range(21, 121):
    ann[f"ann_m{i}"] = ann['ann_p0'] - i*BusinessDay()
    
# set NA values (becuase None is the missing value for strings in Pandas)
ann.replace({'nan':None}, inplace=True)

# Print some basic information
print(f"There are {len(ann.GVKEY.unique())} unique firms")
print(" ") # empty line
print(f"{len(ann.loc[(ann.SEDOL.isnull()==False) | (ann.CUSIP.isnull()==False)].GVKEY.unique())} unique firms have either CUSIP or SEDOL number")
print(f"{len(ann.loc[(ann.SEDOL.isnull()) & (ann.CUSIP.isnull())].GVKEY.unique())} unique firms have neither CUSIP nor SEDOL number")
print(" ") # empty line
print(f"{len(ann[ann.CUSIP.isnull()==False].GVKEY.unique())} unique firms have the CUSIP number")
print(f"{len(ann[ann.SEDOL.isnull()==False].GVKEY.unique())} unique firms have the SEDOL number")
print(" ") # empty line
# print("The GVKEYs for firms with neither CUSIP nor SEDOL numbers are:")
# for i in ann.loc[(ann.SEDOL.isnull()) & (ann.CUSIP.isnull())].GVKEY.unique():
#     print(i)

There are 25201 unique firms
 
25193 unique firms have either CUSIP or SEDOL number
8 unique firms have neither CUSIP nor SEDOL number
 
7950 unique firms have the CUSIP number
17250 unique firms have the SEDOL number
 


In [69]:
# Original data duplicates
ann_noduplicates = ann.drop_duplicates()
ann_noduplicates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161162 entries, 0 to 165515
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   FYEAR        161162 non-null  float64       
 1   GVKEY        161162 non-null  object        
 2   DATADATE     161162 non-null  datetime64[ns]
 3   CONM         161162 non-null  object        
 4   SEDOL        114081 non-null  object        
 5   CUSIP        46990 non-null   object        
 6   ANNDATS_ACT  161162 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), object(4)
memory usage: 9.8+ MB


#### 1.2 The matchable firm list
Note: the following 9 lists are created in Datastream (DS) according to the following process:
1. export the unique firms with either CUSIP or SEDOL number to excel; 
2. divide the firms with codes into sub-groups (with each group less than 5,000 firms becuase DS does not seem to allow larger lists)
3. query the lists separately using 'static requests' and then delete those with no matches
4. save all the firms that are matched this way
5. for those not matched, I use the following strategy: I statically query (i.e. not time series) all active and dead (delisted) firms via DS, and then extract the CUSIP (for U.S. and Canada listed firms) and SEDOL codes. Then, I match the previously unmatched firms to this large list.
6. finally, all the matchable firms (as the later codes show, there are 23,417 of them) are allocated into the 9 lists below (ready for DS query)

- L#XJ01
- L#XJ02
- L#XJ03
- L#XJ04
- L#XJ05
- L#XJ06
- L#XJ07
- L#XJ08
- L#XJ09

### Step 2: Query the lists in Datastream (via Excel add-on) - this process is preformed in Excel
- For each list for firms, I query the daily stock prices, daily market value, and daily trading volume, from 1/1/1998 to 05/31/2021 for each firm
- The process is very time-consuming, as the DS query runs VERY slow using the excel add-in.
- Essentially, the result is 27 files (3 types of query multiplied by 9 lists)

### Step 3: match the queried lists to the sample - Extremly time-consuming
- In this step, I match the original file to the 27 files.
- For each list, the price, market value, and trading volume files are matched together for all the days (i.e., day 2, 1, 0, -1, -2, -21, -22, ..., -120)
- The process is extremely time-consuming considering the number of matching algorithms performed.

#### 3.1 SEDOL List XJ01

In [9]:
sedol_xj01_P = pd.read_csv("DS query results\\DS_XJ01.csv")
sedol_xj01_P['SEDOL']=sedol_xj01_P['Code'].str[:7]
sedol_xj01_P.drop(['Name', 'Code', 'Sedol', 'CURRENCY'], axis=1, inplace=True)

sedol_xj01_P_long = pd.melt(sedol_xj01_P, id_vars=['SEDOL'], var_name = 'date', value_name='price')
sedol_xj01_P_long = sedol_xj01_P_long[sedol_xj01_P_long.SEDOL.isnull()==False]
sedol_xj01_P_long['date'] = pd.to_datetime(sedol_xj01_P_long['date'])
sedol_xj01_P_long.head()

,SEDOL,date,price
0,5165294,1998-01-01,6.6567
1,0287580,1998-01-01,324.05
2,0108120,1998-01-01,1001
4,0798059,1998-01-01,400
5,3091357,1998-01-01,335.51


In [17]:
sedol_xj01_MV = pd.read_csv("DS query results\\DS_XJ01_MV.csv")
sedol_xj01_MV['SEDOL']=sedol_xj01_MV['Code'].str[:7]
sedol_xj01_MV.drop(['Name', 'Code', 'CURRENCY'], axis=1, inplace=True)

sedol_xj01_MV_long = pd.melt(sedol_xj01_MV, id_vars=['SEDOL'], var_name = 'date', value_name='MV')
sedol_xj01_MV_long = sedol_xj01_MV_long[sedol_xj01_MV_long.SEDOL.isnull()==False]
sedol_xj01_MV_long['date'] = pd.to_datetime(sedol_xj01_MV_long['date'])
sedol_xj01_MV_long.head()

,SEDOL,date,MV
0,5165294,1998-01-01,246.37
1,0287580,1998-01-01,17207.23
2,0108120,1998-01-01,4874.87
4,0798059,1998-01-01,46096
5,3091357,1998-01-01,30638.35


In [18]:
sedol_xj01_VO = pd.read_csv("DS query results\\DS_XJ01_VO.csv")
sedol_xj01_VO['SEDOL']=sedol_xj01_VO['Code'].str[:7]
sedol_xj01_VO.drop(['Name', 'Code', 'CURRENCY'], axis=1, inplace=True)

sedol_xj01_VO_long = pd.melt(sedol_xj01_VO, id_vars=['SEDOL'], var_name = 'date', value_name='VO')
sedol_xj01_VO_long = sedol_xj01_VO_long[sedol_xj01_VO_long.SEDOL.isnull()==False]
sedol_xj01_VO_long['date'] = pd.to_datetime(sedol_xj01_VO_long['date'])
sedol_xj01_VO_long.head()

,SEDOL,date,VO
0,5165294,1998-01-01,NaN
1,0287580,1998-01-01,NaN
2,0108120,1998-01-01,NaN
4,0798059,1998-01-01,NaN
5,3091357,1998-01-01,NaN


In [19]:
# Merge
sedol_xj01_long = pd.merge(sedol_xj01_P_long, sedol_xj01_MV_long, on=['SEDOL', 'date'], how='left')
sedol_xj01_long = pd.merge(sedol_xj01_long, sedol_xj01_VO_long, on=['SEDOL', 'date'], how='left')
sedol_xj01_long.head()

,SEDOL,date,price,MV,VO
0,5165294,1998-01-01,6.6567,246.37,NaN
1,0287580,1998-01-01,324.05,17207.23,NaN
2,0108120,1998-01-01,1001,4874.87,NaN
3,0798059,1998-01-01,400,46096,NaN
4,3091357,1998-01-01,335.51,30638.35,NaN


In [22]:
# use for loop to merge to the sample
cols = ann.columns[6:]
ann1 = ann.copy()
for i in cols:
    ann1 = pd.merge(ann1, sedol_xj01_long, left_on=['SEDOL', i], right_on=['SEDOL', 'date'], how='left')
    ann1.rename(columns={'price':f"price_{i[4:]}", 'MV':f"mv_{i[4:]}", 'VO':f"vo_{i[4:]}"}, inplace=True)
    ann1.drop(['date'], axis=1, inplace=True)

In [81]:
# Print out the matching results information
# ann1 = pd.read_pickle("ann1.pkl")
ann1_matched = ann1[(ann1.price_p0.isnull()==False) | 
                    (ann1.price_p1.isnull()==False) | 
                    (ann1.price_m1.isnull()==False)]
ann1_unmatched = ann1[(ann1.price_p0.isnull()) & 
                      (ann1.price_p1.isnull()) & 
                      (ann1.price_m1.isnull()) ]

ann1_matched_gvkeys = list(ann1_matched.GVKEY.unique())
ann1_unmatched_error = ann1_unmatched[ann1_unmatched.GVKEY.isin(ann1_matched_gvkeys)]
ann1_todelete = list(ann1_unmatched_error.GVKEY.unique())
ann1_unmatched = ann1_unmatched[ann1_unmatched.GVKEY.isin(ann1_todelete)==False]

print(f"There are {ann1_unmatched_error.shape[0]} firms unmatched that also appear in the matched list due to lost of data because of listing gaps,\n")
print(f"The number of matched stocks is {len(ann1_matched.GVKEY.unique())}")
print(f"The number of unmatched stocks is {len(ann1_unmatched.GVKEY.unique())}")
ann1_matched.sample(3)

There are 86 firms unmatched that also appear in the matched list due to lost of data because of listing gaps,

The number of matched stocks is 4722
The number of unmatched stocks is 20479


,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,vo_m117,price_m118,mv_m118,vo_m118,price_m119,mv_m119,vo_m119,price_m120,mv_m120,vo_m120
43733,2003.0,101605,2003-06-30,REECE LTD,6728801,None,2003-08-29,2003-09-01,2003-09-02,2003-08-28,...,NaN,1.2378,627.48,15.3,1.2279,622.5,9.2,1.2279,622.5,NaN
80467,2017.0,205649,2018-01-31,YINSON HLDGS,6986717,None,2018-03-29,2018-03-30,2018-04-02,2018-03-28,...,5120.6,1.573,3955.93,4417.0,1.56,3923.15,579.6,1.565,3934.07,2338.2
77846,2008.0,204130,2009-03-31,NIPPON DENSETSU KOGYO CO LTD,6640325,None,2009-05-11,2009-05-12,2009-05-13,2009-05-08,...,158.0,919.0,60044.86,75.0,907.0,59260.84,222.0,910.0,59456.84,NaN


In [24]:
ann1.to_pickle("ann1.pkl")

#### SEDOL List XJ02

In [26]:
# Price
sedol_xj02_P = pd.read_csv("DS query results\\DS_XJ02.csv")
sedol_xj02_P['SEDOL']=sedol_xj02_P['Code'].str[:7]
sedol_xj02_P.drop(['Name', 'Code', 'CURRENCY'], axis=1, inplace=True)

sedol_xj02_P_long = pd.melt(sedol_xj02_P, id_vars=['SEDOL'], var_name = 'date', value_name='price')
sedol_xj02_P_long = sedol_xj02_P_long[sedol_xj02_P_long.SEDOL.isnull()==False]
sedol_xj02_P_long['date'] = pd.to_datetime(sedol_xj02_P_long['date'])

# MV
sedol_xj02_MV = pd.read_csv("DS query results\\DS_XJ02_MV.csv")
sedol_xj02_MV['SEDOL']=sedol_xj02_MV['Code'].str[:7]
sedol_xj02_MV.drop(['Name', 'Code', 'CURRENCY'], axis=1, inplace=True)

sedol_xj02_MV_long = pd.melt(sedol_xj02_MV, id_vars=['SEDOL'], var_name = 'date', value_name='MV')
sedol_xj02_MV_long = sedol_xj02_MV_long[sedol_xj02_MV_long.SEDOL.isnull()==False]
sedol_xj02_MV_long['date'] = pd.to_datetime(sedol_xj02_MV_long['date'])

# Volume
sedol_xj02_VO = pd.read_csv("DS query results\\DS_XJ02_VO.csv")
sedol_xj02_VO['SEDOL']=sedol_xj02_VO['Code'].str[:7]
sedol_xj02_VO.drop(['Name', 'Code', 'CURRENCY'], axis=1, inplace=True)

sedol_xj02_VO_long = pd.melt(sedol_xj02_VO, id_vars=['SEDOL'], var_name = 'date', value_name='VO')
sedol_xj02_VO_long = sedol_xj02_VO_long[sedol_xj02_VO_long.SEDOL.isnull()==False]
sedol_xj02_VO_long['date'] = pd.to_datetime(sedol_xj02_VO_long['date'])

In [27]:
# Merge
sedol_xj02_long = pd.merge(sedol_xj02_P_long, sedol_xj02_MV_long, on=['SEDOL', 'date'], how='left')
sedol_xj02_long = pd.merge(sedol_xj02_long, sedol_xj02_VO_long, on=['SEDOL', 'date'], how='left')
sedol_xj02_long.head()

,SEDOL,date,price,MV,VO
0,5468346,1998-01-01,4.58,274.82,NaN
1,5109560,1998-01-01,25.05,120.26,NaN
2,5182282,1998-01-01,74.1361,142.06,NaN
3,5060322,1998-01-01,8.155,24.47,NaN
4,5970614,1998-01-01,3.66,46.87,NaN


In [28]:
# prepare the table to match
ann1_unmatched.drop(list(ann1_unmatched.filter(regex='mv')), axis=1, inplace=True)
ann1_unmatched.drop(list(ann1_unmatched.filter(regex='price')), axis=1, inplace=True)
ann1_unmatched.drop(list(ann1_unmatched.filter(regex='vo')), axis=1, inplace=True)
ann1_unmatched.drop(list(ann1_unmatched.filter(regex='index')), axis=1, inplace=True)
ann1_unmatched.drop(list(ann1_unmatched.filter(regex='currency')), axis=1, inplace=True)
ann1_unmatched.head()
len(ann1_unmatched.GVKEY.unique())

# use for loop to merge to the sample
cols = ann.columns[6:]
ann2 = ann1_unmatched.copy()
for i in cols:
    ann2 = pd.merge(ann2, sedol_xj02_long, left_on=['SEDOL', i], right_on=['SEDOL', 'date'], how='left')
    ann2.rename(columns={'price':f"price_{i[4:]}", 'MV':f"mv_{i[4:]}", 'VO':f"vo_{i[4:]}"}, inplace=True)
    ann2.drop(['date'], axis=1, inplace=True)

In [82]:
# Print out the matching results information
# ann2 = pd.read_pickle("ann2.pkl")
ann2_matched = ann2[(ann2.price_p0.isnull()==False) | 
                    (ann2.price_p1.isnull()==False) | 
                    (ann2.price_m1.isnull()==False)]
ann2_unmatched = ann2[(ann2.price_p0.isnull()) & 
                      (ann2.price_p1.isnull()) & 
                      (ann2.price_m1.isnull()) ]

ann2_matched_gvkeys = list(ann2_matched.GVKEY.unique())
ann2_unmatched_error = ann2_unmatched[ann2_unmatched.GVKEY.isin(ann2_matched_gvkeys)]
ann2_todelete = list(ann2_unmatched_error.GVKEY.unique())
ann2_unmatched = ann2_unmatched[ann2_unmatched.GVKEY.isin(ann2_todelete)==False]

print(f"There are {ann2_unmatched_error.shape[0]} firms unmatched that also appear in the matched list due to lost of data because of listing gaps,\n")
print(f"The number of matched stocks is {len(ann2_matched.GVKEY.unique())}")
print(f"The number of unmatched stocks is {len(ann2_unmatched.GVKEY.unique())}")
ann2_matched.sample(3)

There are 41 firms unmatched that also appear in the matched list due to lost of data because of listing gaps,

The number of matched stocks is 4249
The number of unmatched stocks is 16230


,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,vo_m117,price_m118,mv_m118,vo_m118,price_m119,mv_m119,vo_m119,price_m120,mv_m120,vo_m120
57897,2000.0,227225,2000-12-31,EASTERN ASIA TECHNOLOGY LTD,6133922,None,2001-08-27,2001-08-28,2001-08-29,2001-08-24,...,2039.4,0.39,140.8,1070.8,0.39,141.9,1646.7,0.4,145.2,1659.9
57668,2016.0,226805,2016-12-31,CENIT AG,5761517,None,2017-03-31,2017-04-03,2017-04-04,2017-03-30,...,1.9,22.171,185.52,0.5,22.86,191.29,11.5,20.21,169.11,0.8
86708,2011.0,271493,2011-06-30,INDEX MULTIMEDIA,7342330,None,2011-11-03,2011-11-04,2011-11-07,2011-11-02,...,3.4,1.3,17.49,0.2,1.3,17.49,1.2,1.29,17.36,0.0


In [30]:
ann2.to_pickle("ann2.pkl")

### SEDOL List XJ03

In [31]:
# Price
sedol_xj03_P = pd.read_csv("DS query results\\DS_XJ03.csv")
sedol_xj03_P['SEDOL']=sedol_xj03_P['Code'].str[:7]
sedol_xj03_P.drop(['Name', 'Code', 'CURRENCY'], axis=1, inplace=True)

sedol_xj03_P_long = pd.melt(sedol_xj03_P, id_vars=['SEDOL'], var_name = 'date', value_name='price')
sedol_xj03_P_long = sedol_xj03_P_long[sedol_xj03_P_long.SEDOL.isnull()==False]
sedol_xj03_P_long['date'] = pd.to_datetime(sedol_xj03_P_long['date'])

# MV
sedol_xj03_MV = pd.read_csv("DS query results\\DS_XJ03_MV.csv")
sedol_xj03_MV['SEDOL']=sedol_xj03_MV['Code'].str[:7]
sedol_xj03_MV.drop(['Name', 'Code', 'CURRENCY'], axis=1, inplace=True)

sedol_xj03_MV_long = pd.melt(sedol_xj03_MV, id_vars=['SEDOL'], var_name = 'date', value_name='MV')
sedol_xj03_MV_long = sedol_xj03_MV_long[sedol_xj03_MV_long.SEDOL.isnull()==False]
sedol_xj03_MV_long['date'] = pd.to_datetime(sedol_xj03_MV_long['date'])

# Volume
sedol_xj03_VO = pd.read_csv("DS query results\\DS_XJ03_VO.csv")
sedol_xj03_VO['SEDOL']=sedol_xj03_VO['Code'].str[:7]
sedol_xj03_VO.drop(['Name', 'Code', 'CURRENCY'], axis=1, inplace=True)

sedol_xj03_VO_long = pd.melt(sedol_xj03_VO, id_vars=['SEDOL'], var_name = 'date', value_name='VO')
sedol_xj03_VO_long = sedol_xj03_VO_long[sedol_xj03_VO_long.SEDOL.isnull()==False]
sedol_xj03_VO_long['date'] = pd.to_datetime(sedol_xj03_VO_long['date'])

In [32]:
# Merge
sedol_xj03_long = pd.merge(sedol_xj03_P_long, sedol_xj03_MV_long, on=['SEDOL', 'date'], how='left')
sedol_xj03_long = pd.merge(sedol_xj03_long, sedol_xj03_VO_long, on=['SEDOL', 'date'], how='left')
sedol_xj03_long.head()

,SEDOL,date,price,MV,VO
0,6433990,1998-01-01,NaN,NaN,NaN
1,5943401,1998-01-01,NaN,NaN,NaN
2,2780322,1998-01-01,NaN,NaN,NaN
3,6527451,1998-01-01,NaN,NaN,NaN
4,6541774,1998-01-01,NaN,NaN,NaN


In [33]:
# prepare the table to match
ann2_unmatched.drop(list(ann2_unmatched.filter(regex='mv')), axis=1, inplace=True)
ann2_unmatched.drop(list(ann2_unmatched.filter(regex='price')), axis=1, inplace=True)
ann2_unmatched.drop(list(ann2_unmatched.filter(regex='vo')), axis=1, inplace=True)
ann2_unmatched.drop(list(ann2_unmatched.filter(regex='index')), axis=1, inplace=True)
ann2_unmatched.drop(list(ann2_unmatched.filter(regex='currency')), axis=1, inplace=True)
ann2_unmatched.head()
len(ann2_unmatched.GVKEY.unique())

# use for loop to merge to the sample
cols = ann.columns[6:]
ann3 = ann2_unmatched.copy()
for i in cols:
    ann3 = pd.merge(ann3, sedol_xj03_long, left_on=['SEDOL', i], right_on=['SEDOL', 'date'], how='left')
    ann3.rename(columns={'price':f"price_{i[4:]}", 'MV':f"mv_{i[4:]}", 'VO':f"vo_{i[4:]}"}, inplace=True)
    ann3.drop(['date'], axis=1, inplace=True)

In [83]:
# Print out the matching results information
# ann3 = pd.read_pickle("ann3.pkl")
ann3_matched = ann3[(ann3.price_p0.isnull()==False) | 
                    (ann3.price_p1.isnull()==False) | 
                    (ann3.price_m1.isnull()==False)]
ann3_unmatched = ann3[(ann3.price_p0.isnull()) & 
                      (ann3.price_p1.isnull()) & 
                      (ann3.price_m1.isnull()) ]

ann3_matched_gvkeys = list(ann3_matched.GVKEY.unique())
ann3_unmatched_error = ann3_unmatched[ann3_unmatched.GVKEY.isin(ann3_matched_gvkeys)]
ann3_todelete = list(ann3_unmatched_error.GVKEY.unique())
ann3_unmatched = ann3_unmatched[ann3_unmatched.GVKEY.isin(ann3_todelete)==False]

print(f"There are {ann3_unmatched_error.shape[0]} firms unmatched that also appear in the matched list due to lost of data because of listing gaps,\n")
print(f"The number of matched stocks is {len(ann3_matched.GVKEY.unique())}")
print(f"The number of unmatched stocks is {len(ann3_unmatched.GVKEY.unique())}")
ann3_matched.sample(3)

There are 2 firms unmatched that also appear in the matched list due to lost of data because of listing gaps,

The number of matched stocks is 119
The number of unmatched stocks is 16111


,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,vo_m117,price_m118,mv_m118,vo_m118,price_m119,mv_m119,vo_m119,price_m120,mv_m120,vo_m120
62488,2012.0,277923,2012-12-31,AUDAX RENOVABLES SA,7586945,None,2013-03-01,2013-03-04,2013-03-05,2013-02-28,...,229.9,0.48,67.2,874.4,0.49,68.6,1888.0,0.42,58.8,394.1
65972,2012.0,282802,2012-12-31,COSMAX BTI INC,6514334,None,2013-02-13,2013-02-14,2013-02-15,2013-02-12,...,285.8,29437.46,516522.9,408.8,28895.20,507008.1,393.2,28895.20,507008.1,343.5
68819,2018.0,286739,2018-12-31,SPG CO LTD,6541945,None,2019-03-07,2019-03-08,2019-03-11,2019-03-06,...,NaN,8610.00,179750.7,NaN,8610.00,179750.7,318.9,8800.00,183717.3,470.6


In [35]:
ann3.to_pickle("ann3.pkl")

#### SEDOL List XJ04

In [19]:
# Price
sedol_xj04_P = pd.read_csv("DS query results\\DS_XJ04.csv")
sedol_xj04_ref = sedol_xj04_P[['Code2', 'SEDOL']]
sedol_xj04_P.drop(['Name', 'Code', 'Code2', 'CURRENCY'], axis=1, inplace=True)

sedol_xj04_P_long = pd.melt(sedol_xj04_P, id_vars=['SEDOL'], var_name = 'date', value_name='price')
sedol_xj04_P_long = sedol_xj04_P_long[sedol_xj04_P_long.SEDOL.isnull()==False]
sedol_xj04_P_long['date'] = pd.to_datetime(sedol_xj04_P_long['date'])

# # MV
sedol_xj04_MV = pd.read_csv("DS query results\\DS_XJ04_MV.csv")
sedol_xj04_MV = sedol_xj04_MV[sedol_xj04_MV['Code'].isnull()==False]
sedol_xj04_MV['text_location'] = sedol_xj04_MV['Code'].str.find("(")
sedol_xj04_MV['Code2'] = sedol_xj04_MV.apply(lambda x: x['Code'][:x['text_location']], axis=1)
sedol_xj04_MV = pd.merge(sedol_xj04_MV, sedol_xj04_ref, on=['Code2'], how='left')
sedol_xj04_MV.drop(['Name', 'Code', 'Code2', 'text_location', 'CURRENCY'], axis=1, inplace=True)

sedol_xj04_MV_long = pd.melt(sedol_xj04_MV, id_vars=['SEDOL'], var_name = 'date', value_name='MV')
sedol_xj04_MV_long = sedol_xj04_MV_long[sedol_xj04_MV_long.SEDOL.isnull()==False]
sedol_xj04_MV_long['date'] = pd.to_datetime(sedol_xj04_MV_long['date'])

# # Volume
sedol_xj04_VO = pd.read_csv("DS query results\\DS_XJ04_VO.csv")
sedol_xj04_VO = sedol_xj04_VO[sedol_xj04_VO['Code'].isnull()==False]
sedol_xj04_VO['text_location'] = sedol_xj04_VO['Code'].str.find("(").astype(int)
sedol_xj04_VO['Code2'] = sedol_xj04_VO.apply(lambda x: x['Code'][:x['text_location']], axis=1)
sedol_xj04_VO = pd.merge(sedol_xj04_VO, sedol_xj04_ref, on=['Code2'], how='left')
sedol_xj04_VO.drop(['Name', 'Code', 'Code2', 'text_location', 'CURRENCY'], axis=1, inplace=True)

sedol_xj04_VO_long = pd.melt(sedol_xj04_VO, id_vars=['SEDOL'], var_name = 'date', value_name='VO')
sedol_xj04_VO_long = sedol_xj04_VO_long[sedol_xj04_VO_long.SEDOL.isnull()==False]
sedol_xj04_VO_long['date'] = pd.to_datetime(sedol_xj04_VO_long['date'])

In [23]:
# Merge
sedol_xj04_long = pd.merge(sedol_xj04_P_long, sedol_xj04_MV_long, on=['SEDOL', 'date'], how='left')
sedol_xj04_long = pd.merge(sedol_xj04_long, sedol_xj04_VO_long, on=['SEDOL', 'date'], how='left')
sedol_xj04_long.head()

,SEDOL,date,price,MV,VO
0,BD0R0N4,1998-01-01,1.20,174.87,NaN
1,BDGN274,1998-01-01,880.00,95681.44,NaN
2,B28ZPV6,1998-01-01,6.51,3.81,NaN
3,B10RZP7,1998-01-01,1149.88,17005.43,NaN
4,B11HK39,1998-01-01,19.52,82351.69,NaN


In [25]:
# prepare the table to match
ann3_unmatched.drop(list(ann3_unmatched.filter(regex='mv')), axis=1, inplace=True)
ann3_unmatched.drop(list(ann3_unmatched.filter(regex='price')), axis=1, inplace=True)
ann3_unmatched.drop(list(ann3_unmatched.filter(regex='vo')), axis=1, inplace=True)
ann3_unmatched.drop(list(ann3_unmatched.filter(regex='index')), axis=1, inplace=True)
ann3_unmatched.drop(list(ann3_unmatched.filter(regex='currency')), axis=1, inplace=True)
ann3_unmatched.head()
len(ann3_unmatched.GVKEY.unique())

# use for loop to merge to the sample
cols = ann.columns[6:]
ann4 = ann3_unmatched.copy()
for i in cols:
    ann4 = pd.merge(ann4, sedol_xj04_long, left_on=['SEDOL', i], right_on=['SEDOL', 'date'], how='left')
    ann4.rename(columns={'price':f"price_{i[4:]}", 'MV':f"mv_{i[4:]}", 'VO':f"vo_{i[4:]}"}, inplace=True)
    ann4.drop(['date'], axis=1, inplace=True)

In [84]:
# Print out the matching results information
# ann4 = pd.read_pickle("ann4.pkl")
ann4_matched = ann4[(ann4.price_p0.isnull()==False) | 
                    (ann4.price_p1.isnull()==False) | 
                    (ann4.price_m1.isnull()==False)]
ann4_unmatched = ann4[(ann4.price_p0.isnull()) & 
                      (ann4.price_p1.isnull()) & 
                      (ann4.price_m1.isnull()) ]

ann4_matched_gvkeys = list(ann4_matched.GVKEY.unique())
ann4_unmatched_error = ann4_unmatched[ann4_unmatched.GVKEY.isin(ann4_matched_gvkeys)]
ann4_todelete = list(ann4_unmatched_error.GVKEY.unique())
ann4_unmatched = ann4_unmatched[ann4_unmatched.GVKEY.isin(ann4_todelete)==False]

print(f"There are {ann4_unmatched_error.shape[0]} firms unmatched that also appear in the matched list due to lost of data because of listing gaps,\n")
print(f"The number of matched stocks is {len(ann4_matched.GVKEY.unique())}")
print(f"The number of unmatched stocks is {len(ann4_unmatched.GVKEY.unique())}")
ann4_matched.sample(3)

There are 30 firms unmatched that also appear in the matched list due to lost of data because of listing gaps,

The number of matched stocks is 977
The number of unmatched stocks is 15134


,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,vo_m117,price_m118,mv_m118,vo_m118,price_m119,mv_m119,vo_m119,price_m120,mv_m120,vo_m120
49493,2015.0,202564,2015-12-31,GSS ENERGY LTD,BV9FNF8,None,2016-02-29,2016-03-01,2016-03-02,2016-02-26,...,2492.9,0.081,38.28,79.5,0.074,34.97,292.1,0.078,36.86,1236.1
55393,2019.0,270732,2019-12-31,SHAANXI CONST MACHINERY CO,B01T191,None,2020-04-27,2020-04-28,2020-04-29,2020-04-24,...,11068.1,7.420,7988.20,15321.9,7.460,8029.59,16912.7,7.810,8402.10,6093.3
54601,2020.0,270384,2020-12-31,SHANGHAI PUDONG CONSTRUCTION,B00FR58,None,2021-03-15,2021-03-16,2021-03-17,2021-03-12,...,NaN,6.190,6005.88,2312.8,6.200,6015.58,2656.0,6.190,6005.88,2671.9


In [27]:
ann4.to_pickle("ann4.pkl")

#### SEDOL List XJ05

In [28]:
# Price
sedol_xj05_P = pd.read_csv("DS query results\\DS_XJ05.csv")
sedol_xj05_ref = sedol_xj05_P[['Code', 'SEDOL']]
sedol_xj05_P.drop(['Name', 'Code', 'CURRENCY'], axis=1, inplace=True)

sedol_xj05_P_long = pd.melt(sedol_xj05_P, id_vars=['SEDOL'], var_name = 'date', value_name='price')
sedol_xj05_P_long = sedol_xj05_P_long[sedol_xj05_P_long.SEDOL.isnull()==False]
sedol_xj05_P_long['date'] = pd.to_datetime(sedol_xj05_P_long['date'])

# # MV
sedol_xj05_MV = pd.read_csv("DS query results\\DS_XJ05_MV.csv")
sedol_xj05_MV = sedol_xj05_MV[sedol_xj05_MV['Code'].isnull()==False]
sedol_xj05_MV['text_location'] = sedol_xj05_MV['Code'].str.find("(")
sedol_xj05_MV['Code'] = sedol_xj05_MV.apply(lambda x: x['Code'][:x['text_location']], axis=1)
sedol_xj05_MV = pd.merge(sedol_xj05_MV, sedol_xj05_ref, on=['Code'], how='left')
sedol_xj05_MV.drop(['Name', 'Code', 'text_location', 'CURRENCY'], axis=1, inplace=True)

sedol_xj05_MV_long = pd.melt(sedol_xj05_MV, id_vars=['SEDOL'], var_name = 'date', value_name='MV')
sedol_xj05_MV_long = sedol_xj05_MV_long[sedol_xj05_MV_long.SEDOL.isnull()==False]
sedol_xj05_MV_long['date'] = pd.to_datetime(sedol_xj05_MV_long['date'])

# # Volume
sedol_xj05_VO = pd.read_csv("DS query results\\DS_XJ05_VO.csv")
sedol_xj05_VO = sedol_xj05_VO[sedol_xj05_VO['Code'].isnull()==False]
sedol_xj05_VO['text_location'] = sedol_xj05_VO['Code'].str.find("(").astype(int)
sedol_xj05_VO['Code'] = sedol_xj05_VO.apply(lambda x: x['Code'][:x['text_location']], axis=1)
sedol_xj05_VO = pd.merge(sedol_xj05_VO, sedol_xj05_ref, on=['Code'], how='left')
sedol_xj05_VO.drop(['Name', 'Code', 'text_location', 'CURRENCY'], axis=1, inplace=True)

sedol_xj05_VO_long = pd.melt(sedol_xj05_VO, id_vars=['SEDOL'], var_name = 'date', value_name='VO')
sedol_xj05_VO_long = sedol_xj05_VO_long[sedol_xj05_VO_long.SEDOL.isnull()==False]
sedol_xj05_VO_long['date'] = pd.to_datetime(sedol_xj05_VO_long['date'])

In [29]:
# Merge
sedol_xj05_long = pd.merge(sedol_xj05_P_long, sedol_xj05_MV_long, on=['SEDOL', 'date'], how='left')
sedol_xj05_long = pd.merge(sedol_xj05_long, sedol_xj05_VO_long, on=['SEDOL', 'date'], how='left')
sedol_xj05_long.head()

,SEDOL,date,price,MV,VO
0,BG0SSL2,1998-01-01,NaN,NaN,NaN
1,B9276C5,1998-01-01,NaN,NaN,NaN
2,B0K2PB1,1998-01-01,NaN,NaN,NaN
3,B0D01C5,1998-01-01,NaN,NaN,NaN
4,B06N217,1998-01-01,NaN,NaN,NaN


In [31]:
# prepare the table to match
ann4_unmatched.drop(list(ann4_unmatched.filter(regex='mv')), axis=1, inplace=True)
ann4_unmatched.drop(list(ann4_unmatched.filter(regex='price')), axis=1, inplace=True)
ann4_unmatched.drop(list(ann4_unmatched.filter(regex='vo')), axis=1, inplace=True)
ann4_unmatched.drop(list(ann4_unmatched.filter(regex='index')), axis=1, inplace=True)
ann4_unmatched.drop(list(ann4_unmatched.filter(regex='currency')), axis=1, inplace=True)
ann4_unmatched.head()
len(ann4_unmatched.GVKEY.unique())

# use for loop to merge to the sample
cols = ann.columns[6:]
ann5 = ann4_unmatched.copy()
for i in cols:
    ann5 = pd.merge(ann5, sedol_xj05_long, left_on=['SEDOL', i], right_on=['SEDOL', 'date'], how='left')
    ann5.rename(columns={'price':f"price_{i[4:]}", 'MV':f"mv_{i[4:]}", 'VO':f"vo_{i[4:]}"}, inplace=True)
    ann5.drop(['date'], axis=1, inplace=True)

In [85]:
# Print out the matching results information
# ann5 = pd.read_pickle("ann5.pkl")
ann5_matched = ann5[(ann5.price_p0.isnull()==False) | 
                    (ann5.price_p1.isnull()==False) | 
                    (ann5.price_m1.isnull()==False)]
ann5_unmatched = ann5[(ann5.price_p0.isnull()) & 
                      (ann5.price_p1.isnull()) & 
                      (ann5.price_m1.isnull()) ]

ann5_matched_gvkeys = list(ann5_matched.GVKEY.unique())
ann5_unmatched_error = ann5_unmatched[ann5_unmatched.GVKEY.isin(ann5_matched_gvkeys)]
ann5_todelete = list(ann5_unmatched_error.GVKEY.unique())
ann5_unmatched = ann5_unmatched[ann5_unmatched.GVKEY.isin(ann5_todelete)==False]

print(f"There are {ann5_unmatched_error.shape[0]} firms unmatched that also appear in the matched list due to lost of data because of listing gaps,\n")
print(f"The number of matched stocks is {len(ann5_matched.GVKEY.unique())}")
print(f"The number of unmatched stocks is {len(ann5_unmatched.GVKEY.unique())}")
ann5_matched.sample(3)

There are 39 firms unmatched that also appear in the matched list due to lost of data because of listing gaps,

The number of matched stocks is 4001
The number of unmatched stocks is 11133


,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,vo_m117,price_m118,mv_m118,vo_m118,price_m119,mv_m119,vo_m119,price_m120,mv_m120,vo_m120
82242,2018.0,329686,2018-12-31,M I TECH CO LTD,BHJWW76,None,2019-02-25,2019-02-26,2019-02-27,2019-02-22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69584,2014.0,296497,2014-06-30,BENALEC HOLDINGS BHD,B65MJQ1,None,2014-08-25,2014-08-26,2014-08-27,2014-08-22,...,660.6,0.86,694.93,820.5,0.86,694.93,4610.8,0.87,703.01,1317.5
74211,2014.0,313685,2014-12-31,INRETAIL PERU CORP,B7YYJD2,None,2015-02-26,2015-02-27,2015-03-02,2015-02-25,...,20.2,18.35,1831.46,4.0,18.45,1841.44,NaN,18.45,1841.44,3.9


In [33]:
ann5.to_pickle("ann5.pkl")

#### SEDOL List XJ06

In [34]:
# Price
sedol_xj06_P = pd.read_csv("DS query results\\DS_XJ06.csv")
sedol_xj06_ref = sedol_xj06_P[['Code', 'SEDOL']]
sedol_xj06_P.drop(['Name', 'Code', 'CURRENCY'], axis=1, inplace=True)

sedol_xj06_P_long = pd.melt(sedol_xj06_P, id_vars=['SEDOL'], var_name = 'date', value_name='price')
sedol_xj06_P_long = sedol_xj06_P_long[sedol_xj06_P_long.SEDOL.isnull()==False]
sedol_xj06_P_long['date'] = pd.to_datetime(sedol_xj06_P_long['date'])

# # MV
sedol_xj06_MV = pd.read_csv("DS query results\\DS_XJ06_MV.csv")
sedol_xj06_MV = sedol_xj06_MV[sedol_xj06_MV['Code'].isnull()==False]
sedol_xj06_MV['text_location'] = sedol_xj06_MV['Code'].str.find("(")
sedol_xj06_MV['Code'] = sedol_xj06_MV.apply(lambda x: x['Code'][:x['text_location']], axis=1)
sedol_xj06_MV = pd.merge(sedol_xj06_MV, sedol_xj06_ref, on=['Code'], how='left')
sedol_xj06_MV.drop(['Name', 'Code', 'text_location', 'CURRENCY'], axis=1, inplace=True)

sedol_xj06_MV_long = pd.melt(sedol_xj06_MV, id_vars=['SEDOL'], var_name = 'date', value_name='MV')
sedol_xj06_MV_long = sedol_xj06_MV_long[sedol_xj06_MV_long.SEDOL.isnull()==False]
sedol_xj06_MV_long['date'] = pd.to_datetime(sedol_xj06_MV_long['date'])

# # Volume
sedol_xj06_VO = pd.read_csv("DS query results\\DS_XJ06_VO.csv")
sedol_xj06_VO = sedol_xj06_VO[sedol_xj06_VO['Code'].isnull()==False]
sedol_xj06_VO['text_location'] = sedol_xj06_VO['Code'].str.find("(").astype(int)
sedol_xj06_VO['Code'] = sedol_xj06_VO.apply(lambda x: x['Code'][:x['text_location']], axis=1)
sedol_xj06_VO = pd.merge(sedol_xj06_VO, sedol_xj06_ref, on=['Code'], how='left')
sedol_xj06_VO.drop(['Name', 'Code', 'text_location', 'CURRENCY'], axis=1, inplace=True)

sedol_xj06_VO_long = pd.melt(sedol_xj06_VO, id_vars=['SEDOL'], var_name = 'date', value_name='VO')
sedol_xj06_VO_long = sedol_xj06_VO_long[sedol_xj06_VO_long.SEDOL.isnull()==False]
sedol_xj06_VO_long['date'] = pd.to_datetime(sedol_xj06_VO_long['date'])

In [35]:
# Merge
sedol_xj06_long = pd.merge(sedol_xj06_P_long, sedol_xj06_MV_long, on=['SEDOL', 'date'], how='left')
sedol_xj06_long = pd.merge(sedol_xj06_long, sedol_xj06_VO_long, on=['SEDOL', 'date'], how='left')
sedol_xj06_long.head()

,SEDOL,date,price,MV,VO
0,3188044,1998-01-01,NaN,NaN,NaN
1,5719981,1998-01-01,NaN,NaN,NaN
2,6013972,1998-01-01,583.7539,603.84,NaN
3,6080523,1998-01-01,NaN,NaN,NaN
4,6107381,1998-01-01,10.4127,10.73,NaN


In [36]:
# prepare the table to match
ann5_unmatched.drop(list(ann5_unmatched.filter(regex='mv')), axis=1, inplace=True)
ann5_unmatched.drop(list(ann5_unmatched.filter(regex='price')), axis=1, inplace=True)
ann5_unmatched.drop(list(ann5_unmatched.filter(regex='vo')), axis=1, inplace=True)
ann5_unmatched.drop(list(ann5_unmatched.filter(regex='index')), axis=1, inplace=True)
ann5_unmatched.drop(list(ann5_unmatched.filter(regex='currency')), axis=1, inplace=True)
ann5_unmatched.head()
len(ann5_unmatched.GVKEY.unique())

# use for loop to merge to the sample
cols = ann.columns[6:]
ann6 = ann5_unmatched.copy()
for i in cols:
    ann6 = pd.merge(ann6, sedol_xj06_long, left_on=['SEDOL', i], right_on=['SEDOL', 'date'], how='left')
    ann6.rename(columns={'price':f"price_{i[4:]}", 'MV':f"mv_{i[4:]}", 'VO':f"vo_{i[4:]}"}, inplace=True)
    ann6.drop(['date'], axis=1, inplace=True)

In [86]:
# Print out the matching results information
# ann6 = pd.read_pickle("ann6.pkl")
ann6_matched = ann6[(ann6.price_p0.isnull()==False) | 
                    (ann6.price_p1.isnull()==False) | 
                    (ann6.price_m1.isnull()==False)]
ann6_unmatched = ann6[(ann6.price_p0.isnull()) & 
                      (ann6.price_p1.isnull()) & 
                      (ann6.price_m1.isnull()) ]

ann6_matched_gvkeys = list(ann6_matched.GVKEY.unique())
ann6_unmatched_error = ann6_unmatched[ann6_unmatched.GVKEY.isin(ann6_matched_gvkeys)]
ann6_todelete = list(ann6_unmatched_error.GVKEY.unique())
ann6_unmatched = ann6_unmatched[ann6_unmatched.GVKEY.isin(ann6_todelete)==False]

print(f"There are {ann6_unmatched_error.shape[0]} firms unmatched that also appear in the matched list due to lost of data because of listing gaps,\n")
print(f"The number of matched stocks is {len(ann6_matched.GVKEY.unique())}")
print(f"The number of unmatched stocks is {len(ann6_unmatched.GVKEY.unique())}")
ann6_matched.sample(3)

There are 71 firms unmatched that also appear in the matched list due to lost of data because of listing gaps,

The number of matched stocks is 2880
The number of unmatched stocks is 8253


,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,vo_m117,price_m118,mv_m118,vo_m118,price_m119,mv_m119,vo_m119,price_m120,mv_m120,vo_m120
55052,2010.0,285021,2010-12-31,TRANSICS INTERNATIONAL NV,B1YMW68,None,2011-02-28,2011-03-01,2011-03-02,2011-02-25,...,0.5,5.25,42.45,12.0,5.16,41.72,5.3,5.19,41.97,2.1
56252,2017.0,289105,2017-12-31,BURKHALTER HOLDING AG,BB97064,None,2018-04-09,2018-04-10,2018-04-11,2018-04-06,...,10.3,115.40,692.19,12.1,117.50,704.78,9.4,118.70,711.98,9.1
53262,2006.0,278193,2007-02-28,CLINPHONE GROUP LTD,B0ZL4M7,None,2008-04-29,2008-04-30,2008-05-01,2008-04-28,...,NaN,64.50,42.84,11.6,64.50,42.84,2215.0,64.50,42.84,77.5


In [38]:
ann6.to_pickle('ann6.pkl')

#### SEDOL List XJ07

In [39]:
# Price
sedol_xj07_P = pd.read_csv("DS query results\\DS_XJ07.csv")
sedol_xj07_ref = sedol_xj07_P[['Code', 'SEDOL']]
sedol_xj07_P.drop(['Name', 'Code', 'CURRENCY'], axis=1, inplace=True)

sedol_xj07_P_long = pd.melt(sedol_xj07_P, id_vars=['SEDOL'], var_name = 'date', value_name='price')
sedol_xj07_P_long = sedol_xj07_P_long[sedol_xj07_P_long.SEDOL.isnull()==False]
sedol_xj07_P_long['date'] = pd.to_datetime(sedol_xj07_P_long['date'])

# # MV
sedol_xj07_MV = pd.read_csv("DS query results\\DS_XJ07_MV.csv")
sedol_xj07_MV = sedol_xj07_MV[sedol_xj07_MV['Code'].isnull()==False]
sedol_xj07_MV['text_location'] = sedol_xj07_MV['Code'].str.find("(")
sedol_xj07_MV['Code'] = sedol_xj07_MV.apply(lambda x: x['Code'][:x['text_location']], axis=1)
sedol_xj07_MV = pd.merge(sedol_xj07_MV, sedol_xj07_ref, on=['Code'], how='left')
sedol_xj07_MV.drop(['Name', 'Code', 'text_location', 'CURRENCY'], axis=1, inplace=True)

sedol_xj07_MV_long = pd.melt(sedol_xj07_MV, id_vars=['SEDOL'], var_name = 'date', value_name='MV')
sedol_xj07_MV_long = sedol_xj07_MV_long[sedol_xj07_MV_long.SEDOL.isnull()==False]
sedol_xj07_MV_long['date'] = pd.to_datetime(sedol_xj07_MV_long['date'])

# # Volume
sedol_xj07_VO = pd.read_csv("DS query results\\DS_XJ07_VO.csv")
sedol_xj07_VO = sedol_xj07_VO[sedol_xj07_VO['Code'].isnull()==False]
sedol_xj07_VO['text_location'] = sedol_xj07_VO['Code'].str.find("(").astype(int)
sedol_xj07_VO['Code'] = sedol_xj07_VO.apply(lambda x: x['Code'][:x['text_location']], axis=1)
sedol_xj07_VO = pd.merge(sedol_xj07_VO, sedol_xj07_ref, on=['Code'], how='left')
sedol_xj07_VO.drop(['Name', 'Code', 'text_location', 'CURRENCY'], axis=1, inplace=True)

sedol_xj07_VO_long = pd.melt(sedol_xj07_VO, id_vars=['SEDOL'], var_name = 'date', value_name='VO')
sedol_xj07_VO_long = sedol_xj07_VO_long[sedol_xj07_VO_long.SEDOL.isnull()==False]
sedol_xj07_VO_long['date'] = pd.to_datetime(sedol_xj07_VO_long['date'])

In [40]:
# Merge
sedol_xj07_long = pd.merge(sedol_xj07_P_long, sedol_xj07_MV_long, on=['SEDOL', 'date'], how='left')
sedol_xj07_long = pd.merge(sedol_xj07_long, sedol_xj07_VO_long, on=['SEDOL', 'date'], how='left')
sedol_xj07_long.head()

,SEDOL,date,price,MV,VO
0,B29MWZ9,1998-01-01,1894.39,11258.45,NaN
1,BG0SSL2,1998-01-01,NaN,NaN,NaN
2,B3KHXB3,1998-01-01,47.21,111.42,NaN
3,B28TMS4,1998-01-01,144.37,187.94,NaN
4,B032D70,1998-01-01,NaN,NaN,NaN


In [41]:
# prepare the table to match
ann6_unmatched.drop(list(ann6_unmatched.filter(regex='mv')), axis=1, inplace=True)
ann6_unmatched.drop(list(ann6_unmatched.filter(regex='price')), axis=1, inplace=True)
ann6_unmatched.drop(list(ann6_unmatched.filter(regex='vo')), axis=1, inplace=True)
ann6_unmatched.drop(list(ann6_unmatched.filter(regex='index')), axis=1, inplace=True)
ann6_unmatched.drop(list(ann6_unmatched.filter(regex='currency')), axis=1, inplace=True)
ann6_unmatched.head()
len(ann6_unmatched.GVKEY.unique())

# use for loop to merge to the sample
cols = ann.columns[6:]
ann7 = ann6_unmatched.copy()
for i in cols:
    ann7 = pd.merge(ann7, sedol_xj07_long, left_on=['SEDOL', i], right_on=['SEDOL', 'date'], how='left')
    ann7.rename(columns={'price':f"price_{i[4:]}", 'MV':f"mv_{i[4:]}", 'VO':f"vo_{i[4:]}"}, inplace=True)
    ann7.drop(['date'], axis=1, inplace=True)

In [87]:
# Print out the matching results information
# ann7 = pd.read_pickle("ann7.pkl")
ann7_matched = ann7[(ann7.price_p0.isnull()==False) | 
                    (ann7.price_p1.isnull()==False) | 
                    (ann7.price_m1.isnull()==False)]
ann7_unmatched = ann7[(ann7.price_p0.isnull()) & 
                      (ann7.price_p1.isnull()) & 
                      (ann7.price_m1.isnull()) ]

ann7_matched_gvkeys = list(ann7_matched.GVKEY.unique())
ann7_unmatched_error = ann7_unmatched[ann7_unmatched.GVKEY.isin(ann7_matched_gvkeys)]
ann7_todelete = list(ann7_unmatched_error.GVKEY.unique())
ann7_unmatched = ann7_unmatched[ann7_unmatched.GVKEY.isin(ann7_todelete)==False]

print(f"There are {ann7_unmatched_error.shape[0]} firms unmatched that also appear in the matched list due to lost of data because of listing gaps,\n")
print(f"The number of matched stocks is {len(ann7_matched.GVKEY.unique())}")
print(f"The number of unmatched stocks is {len(ann7_unmatched.GVKEY.unique())}")
ann7_matched.sample(3)

There are 3 firms unmatched that also appear in the matched list due to lost of data because of listing gaps,

The number of matched stocks is 114
The number of unmatched stocks is 8139


,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,vo_m117,price_m118,mv_m118,vo_m118,price_m119,mv_m119,vo_m119,price_m120,mv_m120,vo_m120
48166,2014.0,316730,2014-06-30,RECALL HOLDINGS LTD,BH23HT3,None,2014-08-24,2014-08-25,2014-08-26,2014-08-22,...,1156.0,4.46,1395.25,1514.3,4.46,1395.25,599.0,4.52,1414.02,1124.0
48032,2009.0,294220,2010-03-31,TALKTALK TELECOM GROUP PLC,B4YCDF5,None,2010-05-11,2010-05-12,2010-05-13,2010-05-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47790,2008.0,279320,2009-02-28,HOME RETAIL GROUP PLC,B19NKB7,None,2009-04-29,2009-04-30,2009-05-01,2009-04-28,...,4140.0,183.25,1607.92,8726.0,182.25,1599.14,10134.0,194.75,1708.82,5190.0


In [43]:
ann7.to_pickle("ann7.pkl")

#### CUSIP List XJ08

In [51]:
cusiplist_xj08 = pd.read_csv("DS_lookup_tables\\Cusip List 2.csv")
cusiplist_xj08['CUSIP'] = cusiplist_xj08['CUSIP'].str[1:]

# Price
cusip_xj08_P = pd.read_csv("DS query results\\DS_XJ08.csv")
cusip_xj08_P = pd.merge(cusip_xj08_P, cusiplist_xj08, left_on=['Code'], right_on=['Check'], how='left')
cusip_xj08_P.drop(['Name', 'Code', 'SEDOL', 'Check','CURRENCY'], axis=1, inplace=True)

cusip_xj08_P_long = pd.melt(cusip_xj08_P, id_vars=['CUSIP'], var_name = 'date', value_name='price')
cusip_xj08_P_long = cusip_xj08_P_long[cusip_xj08_P_long.CUSIP.isnull()==False]
cusip_xj08_P_long['date'] = pd.to_datetime(cusip_xj08_P_long['date'])

# MV
cusip_xj08_MV = pd.read_csv("DS query results\\DS_XJ08_MV.csv")
cusip_xj08_MV = cusip_xj08_MV[cusip_xj08_MV['Code'].isnull()==False]
cusip_xj08_MV['text_location'] = cusip_xj08_MV['Code'].str.find("(")
cusip_xj08_MV['Code'] = cusip_xj08_MV.apply(lambda x: x['Code'][:x['text_location']], axis=1)
cusip_xj08_MV = pd.merge(cusip_xj08_MV, cusiplist_xj08, left_on=['Code'], right_on=['Check'], how='left')
cusip_xj08_MV.drop(['Name', 'Code', 'Check', 'text_location', 'CURRENCY'], axis=1, inplace=True)

cusip_xj08_MV_long = pd.melt(cusip_xj08_MV, id_vars=['CUSIP'], var_name = 'date', value_name='MV')
cusip_xj08_MV_long = cusip_xj08_MV_long[cusip_xj08_MV_long.CUSIP.isnull()==False]
cusip_xj08_MV_long['date'] = pd.to_datetime(cusip_xj08_MV_long['date'])

# Volume
cusip_xj08_VO = pd.read_csv("DS query results\\DS_XJ08_VO.csv")
cusip_xj08_VO = cusip_xj08_VO[cusip_xj08_VO['Code'].isnull()==False]
cusip_xj08_VO['text_location'] = cusip_xj08_VO['Code'].str.find("(").astype(int)
cusip_xj08_VO['Code'] = cusip_xj08_VO.apply(lambda x: x['Code'][:x['text_location']], axis=1)
cusip_xj08_VO = pd.merge(cusip_xj08_VO, cusiplist_xj08, left_on=['Code'], right_on=['Check'], how='left')
cusip_xj08_VO.drop(['Name', 'Code', 'Check', 'text_location', 'CURRENCY'], axis=1, inplace=True)

cusip_xj08_VO_long = pd.melt(cusip_xj08_VO, id_vars=['CUSIP'], var_name = 'date', value_name='VO')
cusip_xj08_VO_long = cusip_xj08_VO_long[cusip_xj08_VO_long.CUSIP.isnull()==False]
cusip_xj08_VO_long['date'] = pd.to_datetime(cusip_xj08_VO_long['date'])

In [52]:
# Merge
cusip_xj08_long = pd.merge(cusip_xj08_P_long, cusip_xj08_MV_long, on=['CUSIP', 'date'], how='left')
cusip_xj08_long = pd.merge(cusip_xj08_long, cusip_xj08_VO_long, on=['CUSIP', 'date'], how='left')
cusip_xj08_long.head()

,CUSIP,date,price,MV,VO
0,020813101,1998-01-01,21.75,302.43,NaN
1,909914103,1998-01-01,25.31,1119.04,NaN
2,125141101,1998-01-01,NaN,NaN,NaN
3,007768104,1998-01-01,3.5,125.67,NaN
4,002444107,1998-01-01,9.22,1625.84,NaN


In [53]:
# prepare the table to match
ann7_unmatched.drop(list(ann7_unmatched.filter(regex='mv')), axis=1, inplace=True)
ann7_unmatched.drop(list(ann7_unmatched.filter(regex='price')), axis=1, inplace=True)
ann7_unmatched.drop(list(ann7_unmatched.filter(regex='vo')), axis=1, inplace=True)
ann7_unmatched.drop(list(ann7_unmatched.filter(regex='index')), axis=1, inplace=True)
ann7_unmatched.drop(list(ann7_unmatched.filter(regex='currency')), axis=1, inplace=True)
ann7_unmatched.head()
len(ann7_unmatched.GVKEY.unique())

# use for loop to merge to the sample
cols = ann.columns[6:]
ann8 = ann7_unmatched.copy()
for i in cols:
    ann8 = pd.merge(ann8, cusip_xj08_long, left_on=['CUSIP', i], right_on=['CUSIP', 'date'], how='left')
    ann8.rename(columns={'price':f"price_{i[4:]}", 'MV':f"mv_{i[4:]}", 'VO':f"vo_{i[4:]}"}, inplace=True)
    ann8.drop(['date'], axis=1, inplace=True)

In [88]:
# Print out the matching results information
# ann8 = pd.read_pickle("ann8.pkl")
ann8_matched = ann8[(ann8.price_p0.isnull()==False) | 
                    (ann8.price_p1.isnull()==False) | 
                    (ann8.price_m1.isnull()==False)]
ann8_unmatched = ann8[(ann8.price_p0.isnull()) & 
                      (ann8.price_p1.isnull()) & 
                      (ann8.price_m1.isnull()) ]

ann8_matched_gvkeys = list(ann8_matched.GVKEY.unique())
ann8_unmatched_error = ann8_unmatched[ann8_unmatched.GVKEY.isin(ann8_matched_gvkeys)]
ann8_todelete = list(ann8_unmatched_error.GVKEY.unique())
ann8_unmatched = ann8_unmatched[ann8_unmatched.GVKEY.isin(ann8_todelete)==False]

print(f"There are {ann8_unmatched_error.shape[0]} firms unmatched that also appear in the matched list due to lost of data because of listing gaps,\n")
print(f"The number of matched stocks is {len(ann8_matched.GVKEY.unique())}")
print(f"The number of unmatched stocks is {len(ann8_unmatched.GVKEY.unique())}")
ann8_matched.sample(3)

There are 11 firms unmatched that also appear in the matched list due to lost of data because of listing gaps,

The number of matched stocks is 4300
The number of unmatched stocks is 3839


,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,vo_m117,price_m118,mv_m118,vo_m118,price_m119,mv_m119,vo_m119,price_m120,mv_m120,vo_m120
43697,2008.0,176361,2008-06-30,ANIMAL HEALTH INTL INC,None,03525N109,2008-09-09,2008-09-10,2008-09-11,2008-09-08,...,116.6,11.69,284.41,91.4,11.63,282.95,78.5,11.66,283.68,91.4
36950,2018.0,137611,2018-12-31,SOHU COM LTD -ADR,None,83410S108,2019-02-01,2019-02-04,2019-02-05,2019-01-31,...,201.4,21.8,848.39,329.3,22.05,858.11,411.9,21.59,840.21,410.2
40819,2005.0,162355,2005-09-30,UNICA CORP,None,904583101,2005-11-02,2005-11-03,2005-11-04,2005-11-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
ann8.to_pickle("ann8.pkl")

#### CUSIP List XJ09

In [61]:
cusiplist_xj09 = pd.read_csv("DS_lookup_tables\\Cusip List 1.csv")
cusiplist_xj09['CUSIP'] = cusiplist_xj09['CUSIP'].str[2:11]

# Price
cusip_xj09_P = pd.read_csv("DS query results\\DS_XJ09.csv")
cusip_xj09_P = pd.merge(cusip_xj09_P, cusiplist_xj09, left_on=['Code'], right_on=['Check'], how='left')
cusip_xj09_P.drop(['Name', 'Code', 'Check','CURRENCY'], axis=1, inplace=True)

cusip_xj09_P_long = pd.melt(cusip_xj09_P, id_vars=['CUSIP'], var_name = 'date', value_name='price')
cusip_xj09_P_long = cusip_xj09_P_long[cusip_xj09_P_long.CUSIP.isnull()==False]
cusip_xj09_P_long['date'] = pd.to_datetime(cusip_xj09_P_long['date'])

# MV
cusip_xj09_MV = pd.read_csv("DS query results\\DS_XJ09_MV.csv")
cusip_xj09_MV = cusip_xj09_MV[cusip_xj09_MV['Code'].isnull()==False]
cusip_xj09_MV['text_location'] = cusip_xj09_MV['Code'].str.find("(")
cusip_xj09_MV['Code'] = cusip_xj09_MV.apply(lambda x: x['Code'][:x['text_location']], axis=1)
cusip_xj09_MV = pd.merge(cusip_xj09_MV, cusiplist_xj09, left_on=['Code'], right_on=['Check'], how='left')
cusip_xj09_MV.drop(['Name', 'Code', 'Check', 'text_location', 'CURRENCY'], axis=1, inplace=True)

cusip_xj09_MV_long = pd.melt(cusip_xj09_MV, id_vars=['CUSIP'], var_name = 'date', value_name='MV')
cusip_xj09_MV_long = cusip_xj09_MV_long[cusip_xj09_MV_long.CUSIP.isnull()==False]
cusip_xj09_MV_long['date'] = pd.to_datetime(cusip_xj09_MV_long['date'])

# Volume
cusip_xj09_VO = pd.read_csv("DS query results\\DS_XJ09_VO.csv")
cusip_xj09_VO = cusip_xj09_VO[cusip_xj09_VO['Code'].isnull()==False]
cusip_xj09_VO['text_location'] = cusip_xj09_VO['Code'].str.find("(").astype(int)
cusip_xj09_VO['Code'] = cusip_xj09_VO.apply(lambda x: x['Code'][:x['text_location']], axis=1)
cusip_xj09_VO = pd.merge(cusip_xj09_VO, cusiplist_xj09, left_on=['Code'], right_on=['Check'], how='left')
cusip_xj09_VO.drop(['Name', 'Code', 'Check', 'text_location', 'CURRENCY'], axis=1, inplace=True)

cusip_xj09_VO_long = pd.melt(cusip_xj09_VO, id_vars=['CUSIP'], var_name = 'date', value_name='VO')
cusip_xj09_VO_long = cusip_xj09_VO_long[cusip_xj09_VO_long.CUSIP.isnull()==False]
cusip_xj09_VO_long['date'] = pd.to_datetime(cusip_xj09_VO_long['date'])

In [62]:
# Merge
cusip_xj09_long = pd.merge(cusip_xj09_P_long, cusip_xj09_MV_long, on=['CUSIP', 'date'], how='left')
cusip_xj09_long = pd.merge(cusip_xj09_long, cusip_xj09_VO_long, on=['CUSIP', 'date'], how='left')
cusip_xj09_long.head()

,CUSIP,date,price,MV,VO
0,000361105,1998-01-01,25.8346,710.44,NaN
1,02376R102,1998-01-01,NaN,NaN,NaN
2,723484101,1998-01-01,42.3750,3589.92,NaN
3,002824100,1998-01-01,14.6555,50099.81,NaN
4,007903107,1998-01-01,8.8750,2516.44,NaN


In [63]:
# prepare the table to match
ann8_unmatched.drop(list(ann8_unmatched.filter(regex='mv')), axis=1, inplace=True)
ann8_unmatched.drop(list(ann8_unmatched.filter(regex='price')), axis=1, inplace=True)
ann8_unmatched.drop(list(ann8_unmatched.filter(regex='vo')), axis=1, inplace=True)
ann8_unmatched.drop(list(ann8_unmatched.filter(regex='index')), axis=1, inplace=True)
ann8_unmatched.drop(list(ann8_unmatched.filter(regex='currency')), axis=1, inplace=True)
ann8_unmatched.head()
len(ann8_unmatched.GVKEY.unique())

# use for loop to merge to the sample
cols = ann.columns[6:]
ann9 = ann8_unmatched.copy()
for i in cols:
    ann9 = pd.merge(ann9, cusip_xj09_long, left_on=['CUSIP', i], right_on=['CUSIP', 'date'], how='left')
    ann9.rename(columns={'price':f"price_{i[4:]}", 'MV':f"mv_{i[4:]}", 'VO':f"vo_{i[4:]}"}, inplace=True)
    ann9.drop(['date'], axis=1, inplace=True)

In [89]:
# Print out the matching results information
# ann9 = pd.read_pickle("ann9.pkl")
ann9_matched = ann9[(ann9.price_p0.isnull()==False) | 
                    (ann9.price_p1.isnull()==False) | 
                    (ann9.price_m1.isnull()==False)]
ann9_unmatched = ann9[(ann9.price_p0.isnull()) & 
                      (ann9.price_p1.isnull()) & 
                      (ann9.price_m1.isnull()) ]

ann9_matched_gvkeys = list(ann9_matched.GVKEY.unique())
ann9_unmatched_error = ann9_unmatched[ann9_unmatched.GVKEY.isin(ann9_matched_gvkeys)]
ann9_todelete = list(ann9_unmatched_error.GVKEY.unique())
ann9_unmatched = ann9_unmatched[ann9_unmatched.GVKEY.isin(ann9_todelete)==False]

print(f"There are {ann9_unmatched_error.shape[0]} firms unmatched that also appear in the matched list due to lost of data because of listing gaps,\n")
print(f"The number of matched stocks is {len(ann9_matched.GVKEY.unique())}")
print(f"The number of unmatched stocks is {len(ann9_unmatched.GVKEY.unique())}")
ann9_matched.sample(3)

There are 64 firms unmatched that also appear in the matched list due to lost of data because of listing gaps,

The number of matched stocks is 2055
The number of unmatched stocks is 1784


,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,vo_m117,price_m118,mv_m118,vo_m118,price_m119,mv_m119,vo_m119,price_m120,mv_m120,vo_m120
11049,2018.0,027703,2018-12-31,AIRGAIN INC,None,00938A104,2019-02-14,2019-02-15,2019-02-18,2019-02-13,...,39.0,11.90,117.72,NaN,11.90,117.72,30.1,11.88,117.52,25.1
7875,2004.0,015050,2005-03-31,HEROUX-DEVTEK INC,None,42774L109,2005-05-26,2005-05-27,2005-05-30,2005-05-25,...,2.4,4.09,110.17,7.0,4.20,113.13,41.8,4.19,112.86,5.1
5911,2007.0,011017,2008-03-31,UNIVERSAL CORP/VA,None,913456109,2008-05-22,2008-05-23,2008-05-26,2008-05-21,...,253.5,52.73,1443.48,249.3,53.37,1461.00,301.8,53.70,1470.04,340.9


In [66]:
ann9.to_pickle('ann9.pkl')

#### Stack the matched firms together

In [21]:
ann1 =pd.read_pickle("ann1.pkl")
ann2 =pd.read_pickle("ann2.pkl")
ann3 =pd.read_pickle("ann3.pkl")
ann4 =pd.read_pickle("ann4.pkl")
ann5 =pd.read_pickle("ann5.pkl")
ann6 =pd.read_pickle("ann6.pkl")
ann7 =pd.read_pickle("ann7.pkl")
ann8 =pd.read_pickle("ann8.pkl")
ann9 =pd.read_pickle("ann9.pkl")

In [22]:
ann1_matched = ann1[(ann1.price_p0.isnull()==False) | (ann1.price_p1.isnull()==False) | (ann1.price_m1.isnull()==False)]
ann2_matched = ann2[(ann2.price_p0.isnull()==False) | (ann2.price_p1.isnull()==False) | (ann2.price_m1.isnull()==False)]
ann3_matched = ann3[(ann3.price_p0.isnull()==False) | (ann3.price_p1.isnull()==False) | (ann3.price_m1.isnull()==False)]
ann4_matched = ann4[(ann4.price_p0.isnull()==False) | (ann4.price_p1.isnull()==False) | (ann4.price_m1.isnull()==False)]
ann5_matched = ann5[(ann5.price_p0.isnull()==False) | (ann5.price_p1.isnull()==False) | (ann5.price_m1.isnull()==False)]
ann6_matched = ann6[(ann6.price_p0.isnull()==False) | (ann6.price_p1.isnull()==False) | (ann6.price_m1.isnull()==False)]
ann7_matched = ann7[(ann7.price_p0.isnull()==False) | (ann7.price_p1.isnull()==False) | (ann7.price_m1.isnull()==False)]
ann8_matched = ann8[(ann8.price_p0.isnull()==False) | (ann8.price_p1.isnull()==False) | (ann8.price_m1.isnull()==False)]
ann9_matched = ann9[(ann9.price_p0.isnull()==False) | (ann9.price_p1.isnull()==False) | (ann9.price_m1.isnull()==False)]

In [23]:
ann1_matched['matched_by'] = 'SEDOL'
ann2_matched['matched_by'] = 'SEDOL'
ann3_matched['matched_by'] = 'SEDOL'
ann4_matched['matched_by'] = 'SEDOL'
ann5_matched['matched_by'] = 'SEDOL'
ann6_matched['matched_by'] = 'SEDOL'
ann7_matched['matched_by'] = 'SEDOL'
ann8_matched['matched_by'] = 'CUSIP'
ann9_matched['matched_by'] = 'CUSIP'

ann_1_9_combine = pd.concat([ann1_matched, ann2_matched, ann3_matched, ann4_matched, ann5_matched, ann6_matched, 
                             ann7_matched, ann8_matched, ann9_matched])

In [24]:
ann_1_9_combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158709 entries, 189 to 24174
Columns: 427 entries, FYEAR to matched_by
dtypes: datetime64[ns](106), float64(1), object(320)
memory usage: 518.2+ MB


In [25]:
ann_1_9_combine = ann_1_9_combine.sort_values(by=['GVKEY', 'ann_p0'])

In [44]:
ann_1_9_combine.head()

,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,mv_m118,vo_m118,price_m119,mv_m119,vo_m119,price_m120,mv_m120,vo_m120,matched_by,Match
0,1998.0,001004,1999-05-31,AAR CORP,None,000361105,1999-06-24,1999-06-25,1999-06-28,1999-06-23,...,592.58,191.5,22.125,613.37,43.7,22.5625,625.5,112.6,CUSIP,000361105
1,1999.0,001004,2000-05-31,AAR CORP,None,000361105,2000-06-28,2000-06-29,2000-06-30,2000-06-27,...,583.98,139.0,21.375,585.7,239.3,21.5625,590.83,523.3,CUSIP,000361105
2,2000.0,001004,2001-05-31,AAR CORP,None,000361105,2001-06-28,2001-06-29,2001-07-02,2001-06-27,...,355.84,NaN,13.25,355.84,121.7,12.25,328.99,48.6,CUSIP,000361105
3,2001.0,001004,2002-05-31,AAR CORP,None,000361105,2002-06-27,2002-06-28,2002-07-01,2002-06-26,...,244.15,155.2,9.55,256.5,115.1,9.7,260.53,42.5,CUSIP,000361105
4,2002.0,001004,2003-05-31,AAR CORP,None,000361105,2003-07-03,2003-07-04,2003-07-07,2003-07-02,...,169.73,NaN,5.33,169.73,36.6,5.46,173.87,27.3,CUSIP,000361105


In [76]:
ann_1_9_combine.to_pickle("ann_1_9_combine.pkl")
ann9_unmatched.to_pickle("ann9_unmatched.pkl")

### Step 4: Add and Match the benchmark index
- In this step, I match each firm (either CUSIP or SEDOL) to the corresponding benchmark index identified by Datastream
- Similarly the benchmark index (there are 45 of them) contains index price from 1/1/1998 to 05/31/2021.
- the index data is matched to earnings annoucement date 0, 1, 2, -1, -2, -21, -22, ...., -120

In [36]:
index_sedol = pd.read_csv("DS_lookup_tables\\Benchmark Index Match_SEDOL.csv")
index_cusip = pd.read_csv("DS_lookup_tables\\Benchmark Index Match_CUSIP.csv")
index_sedol['Match'] = index_sedol['SEDOL'].str.zfill(7)
index_sedol = index_sedol[index_sedol.SEDOL.isnull()==False]
index_cusip['Match'] = index_cusip['ISIN CODE'].str[2:11]
index_cusip.rename(columns={'SEDOL CODE':'SEDOL'}, inplace=True)
index = pd.concat([index_sedol, index_cusip])
index.rename(columns={'Matched_by':'matched_by'}, inplace=True)
index.drop(['Type', 'ISIN CODE', 'SEDOL'], axis=1, inplace=True)
index.drop_duplicates(inplace=True)

In [37]:
index

,matched_by,Index,Match
0,SEDOL,TOTMKNL,5165294
1,SEDOL,TOTMKUK,0287580
2,SEDOL,TOTMKUK,0108120
4,SEDOL,TOTMKUK,0798059
5,SEDOL,TOTMKUK,3091357
...,...,...,...
6377,CUSIP,TOTMKUS,29355M200
6378,CUSIP,TOTMKCN,30041N107
6379,CUSIP,TOTMKUS,36116M106
6380,CUSIP,TOTMKUS,28618M106


In [29]:
ann_1_9_combine['Match'] = np.select([ann_1_9_combine['matched_by']=='SEDOL', ann_1_9_combine['matched_by']=='CUSIP'],
                             [ann_1_9_combine['SEDOL'], ann_1_9_combine['CUSIP']])

In [30]:
ann_1_9_combine.head()

,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,mv_m118,vo_m118,price_m119,mv_m119,vo_m119,price_m120,mv_m120,vo_m120,matched_by,Match
0,1998.0,001004,1999-05-31,AAR CORP,None,000361105,1999-06-24,1999-06-25,1999-06-28,1999-06-23,...,592.58,191.5,22.125,613.37,43.7,22.5625,625.5,112.6,CUSIP,000361105
1,1999.0,001004,2000-05-31,AAR CORP,None,000361105,2000-06-28,2000-06-29,2000-06-30,2000-06-27,...,583.98,139.0,21.375,585.7,239.3,21.5625,590.83,523.3,CUSIP,000361105
2,2000.0,001004,2001-05-31,AAR CORP,None,000361105,2001-06-28,2001-06-29,2001-07-02,2001-06-27,...,355.84,NaN,13.25,355.84,121.7,12.25,328.99,48.6,CUSIP,000361105
3,2001.0,001004,2002-05-31,AAR CORP,None,000361105,2002-06-27,2002-06-28,2002-07-01,2002-06-26,...,244.15,155.2,9.55,256.5,115.1,9.7,260.53,42.5,CUSIP,000361105
4,2002.0,001004,2003-05-31,AAR CORP,None,000361105,2003-07-03,2003-07-04,2003-07-07,2003-07-02,...,169.73,NaN,5.33,169.73,36.6,5.46,173.87,27.3,CUSIP,000361105


In [38]:
ann_combine = pd.merge(ann_1_9_combine, index, left_on=['Match', 'matched_by'], right_on=['Match', 'matched_by'], how='left')
ann_combine.head()

,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,vo_m118,price_m119,mv_m119,vo_m119,price_m120,mv_m120,vo_m120,matched_by,Match,Index
0,1998.0,001004,1999-05-31,AAR CORP,None,000361105,1999-06-24,1999-06-25,1999-06-28,1999-06-23,...,191.5,22.125,613.37,43.7,22.5625,625.5,112.6,CUSIP,000361105,TOTMKUS
1,1999.0,001004,2000-05-31,AAR CORP,None,000361105,2000-06-28,2000-06-29,2000-06-30,2000-06-27,...,139.0,21.375,585.7,239.3,21.5625,590.83,523.3,CUSIP,000361105,TOTMKUS
2,2000.0,001004,2001-05-31,AAR CORP,None,000361105,2001-06-28,2001-06-29,2001-07-02,2001-06-27,...,NaN,13.25,355.84,121.7,12.25,328.99,48.6,CUSIP,000361105,TOTMKUS
3,2001.0,001004,2002-05-31,AAR CORP,None,000361105,2002-06-27,2002-06-28,2002-07-01,2002-06-26,...,155.2,9.55,256.5,115.1,9.7,260.53,42.5,CUSIP,000361105,TOTMKUS
4,2002.0,001004,2003-05-31,AAR CORP,None,000361105,2003-07-03,2003-07-04,2003-07-07,2003-07-02,...,NaN,5.33,169.73,36.6,5.46,173.87,27.3,CUSIP,000361105,TOTMKUS


In [39]:
# Query and add the index data
index_PI = pd.read_csv("DS query results\\DS_INDEX_PI.csv")
index_PI.drop(['Name', 'Code'], axis=1, inplace=True)
index_PI_long = pd.melt(index_PI, id_vars=['Index'], var_name = 'date', value_name='price_index')
index_PI_long = index_PI_long[index_PI_long.Index.isnull()==False]
index_PI_long['date'] = pd.to_datetime(index_PI_long['date'])

In [43]:
# use for loop to merge to the sample
cols = ann.columns[6:]
ann_combine_temp = ann_combine.copy()
for i in cols:
    ann_combine_temp = pd.merge(ann_combine_temp, index_PI_long, left_on=['Index', i], right_on=['Index', 'date'], how='left')
    ann_combine_temp.rename(columns={'price_index':f"price_index_{i[4:]}"}, inplace=True)
    ann_combine_temp.drop(['date'], axis=1, inplace=True)

In [45]:
ann_combine_final = ann_combine_temp.copy()
ann_combine_final.to_pickle("ann_combine_final.pkl")
ann_combine_final.head()

,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,price_index_m111,price_index_m112,price_index_m113,price_index_m114,price_index_m115,price_index_m116,price_index_m117,price_index_m118,price_index_m119,price_index_m120
0,1998.0,001004,1999-05-31,AAR CORP,None,000361105,1999-06-24,1999-06-25,1999-06-28,1999-06-23,...,1142.10,1141.23,1134.62,1134.62,1109.64,1129.41,1136.17,1155.30,1161.73,1158.29
1,1999.0,001004,2000-05-31,AAR CORP,None,000361105,2000-06-28,2000-06-29,2000-06-30,2000-06-27,...,1284.81,1281.40,1314.84,1320.08,1329.28,1326.94,1332.42,1332.42,1318.53,1301.05
2,2000.0,001004,2001-05-31,AAR CORP,None,000361105,2001-06-28,2001-06-29,2001-07-02,2001-06-27,...,1213.43,1208.71,1191.28,1191.18,1197.17,1182.86,1180.17,1172.92,1172.92,1179.45
3,2001.0,001004,2002-05-31,AAR CORP,None,000361105,2002-06-27,2002-06-28,2002-07-01,2002-06-26,...,1013.09,1006.15,1012.51,1012.51,1021.54,1009.79,1027.08,1020.42,1026.60,1034.74
4,2002.0,001004,2003-05-31,AAR CORP,None,000361105,2003-07-03,2003-07-04,2003-07-07,2003-07-02,...,800.34,796.82,787.65,800.38,822.14,815.82,823.28,835.61,835.61,845.71


### Step 5: Perform final calculations

Main Variables created:

- **CAR1**: Cumulative abnormal returns (over DS benchmark index) from -1 to +1
- **CAR2**: Cumulative abnormal returns (over DS benchmark index) from -2 to +2
- **abvo_pm1**: the abnormal trading volume, which is the average trading volume from -1 to +1 over the average trading volume from -21 to -120
- **abvo_p1p0**: the abnormal trading volume, which is the average trading volume from 0 to +1 over the average trading volume from -21 to -120
- **abretvar_overbenchmark_pm1**: the abnormal return variance from -1 to +1 over the abnormal return variance from -21 to -120

In [46]:
# Make sure columns participating in calculations are of the right type
# ann_combine_final = pd.read_pickle("ann_combine_final.pkl")
convert_type_cols = ann_combine_final.columns[111:426]

for i in convert_type_cols:
    ann_combine_final[i] = ann_combine_final[i].astype(float)

In [47]:
# calculate CAR (-1, +1)
ann_combine_final1 = ann_combine_final.copy()
ann_combine_final1['return_pm1'] = ann_combine_final1['price_p1'].div(ann_combine_final['price_m1']).replace(np.inf, np.nan)
ann_combine_final1['return_index_pm1'] = ann_combine_final1['price_index_p1'].div(ann_combine_final1['price_index_m1']).replace(np.inf, np.nan)
ann_combine_final1['CAR1'] = ann_combine_final1['return_pm1'] - ann_combine_final1['return_index_pm1']

# calculate CAR (-2, +2)
ann_combine_final1['return_pm2'] = ann_combine_final1['price_p2'].div(ann_combine_final['price_m2']).replace(np.inf, np.nan)
ann_combine_final1['return_index_pm2'] = ann_combine_final1['price_index_p2'].div(ann_combine_final1['price_index_m2']).replace(np.inf, np.nan)
ann_combine_final1['CAR2'] = ann_combine_final1['return_pm2'] - ann_combine_final1['return_index_pm2']

In [48]:
# Calculate abnormal trading volumes (-1, +1)
ann_combine_final2 = ann_combine_final1.copy()
vo_cols = ann_combine_final2.columns[list(range(128,426,3))]
ann_combine_final2['voavg_pm1'] = ann_combine_final2[['vo_p1', 'vo_p0', 'vo_m1']].mean(axis=1)
ann_combine_final2['voavg_m21m120'] = ann_combine_final2[vo_cols].mean(axis=1)
ann_combine_final2['abvo_pm1'] = ann_combine_final2['voavg_pm1'] / ann_combine_final2['voavg_m21m120']

# Calculate abnormal trading volumes (0, +1)
ann_combine_final2['voavg_p1p0'] = ann_combine_final2[['vo_p1', 'vo_p0']].mean(axis=1)
ann_combine_final2['abvo_p1p0'] = ann_combine_final2['voavg_p1p0'] / ann_combine_final2['voavg_m21m120']

In [49]:
# Glance at the new columns
ann_combine_final2[['vo_m1', 'vo_p0', 'vo_p1', 'voavg_pm1', 'voavg_m21m120', 'abvo_pm1', 'abvo_p1p0','CAR1', 'CAR2']].head(10)

,vo_m1,vo_p0,vo_p1,voavg_pm1,voavg_m21m120,abvo_pm1,abvo_p1p0,CAR1,CAR2
0,82.1,156.4,117.4,118.633333,142.357732,0.833347,0.961662,0.097733,0.203228
1,66.8,994.5,455.8,505.700000,124.523958,4.061066,5.823377,-0.096217,-0.125370
2,24.8,79.4,171.9,92.033333,73.376042,1.254270,1.712412,0.115909,0.025816
3,117.0,204.6,221.5,181.033333,150.108333,1.206018,1.419308,0.100999,-0.086481
4,107.0,54.1,NaN,80.550000,62.444792,1.289939,0.866365,-0.038054,-0.136474
5,104.6,322.5,165.7,197.600000,225.662500,0.875644,1.081704,0.090073,0.067068
6,302.4,772.9,422.9,499.400000,184.591753,2.705430,3.239040,0.018624,-0.008295
7,984.0,3968.5,1224.8,2059.100000,466.903093,4.410123,5.561432,0.039122,0.025711
8,489.8,626.7,571.6,562.700000,558.302062,1.007877,1.073165,-0.007103,-0.003277
9,791.1,1162.6,2223.1,1392.266667,865.968041,1.607758,1.954864,0.010107,0.080261


In [50]:
# Calculate daily abnormal returns and abnormal returns variance
ann_combine_final3 = ann_combine_final2.copy()

for i in range(21, 120):
    ann_combine_final3[f"return_m{i}"] = ann_combine_final3[f"price_m{i}"].div(ann_combine_final3[f"price_m{i+1}"]).replace(np.inf, np.nan)
    ann_combine_final3[f"return_index_m{i}"] = ann_combine_final3[f"price_index_m{i}"].div(ann_combine_final3[f"price_index_m{i+1}"]).replace(np.inf, np.nan)
    ann_combine_final3[f"abret_m{i}"] = ann_combine_final3[f"return_m{i}"] - ann_combine_final3[f"return_index_m{i}"]

ann_combine_final3['return_p1'] = ann_combine_final3['price_p1'].div(ann_combine_final3['price_p0']).replace(np.inf, np.nan)
ann_combine_final3['return_p0'] = ann_combine_final3['price_p0'].div(ann_combine_final3['price_m1']).replace(np.inf, np.nan)
ann_combine_final3['return_m1'] = ann_combine_final3['price_m1'].div(ann_combine_final3['price_m2']).replace(np.inf, np.nan)
ann_combine_final3['return_index_p1'] = ann_combine_final3['price_index_p1'].div(ann_combine_final3['price_index_p0']).replace(np.inf, np.nan)
ann_combine_final3['return_index_p0'] = ann_combine_final3['price_index_p0'].div(ann_combine_final3['price_index_m1']).replace(np.inf, np.nan)
ann_combine_final3['return_index_m1'] = ann_combine_final3['price_index_m1'].div(ann_combine_final3['price_index_m2']).replace(np.inf, np.nan)

ann_combine_final3['abret_p1'] = ann_combine_final3['return_p1'] - ann_combine_final3['return_index_p1']
ann_combine_final3['abret_p0'] = ann_combine_final3['return_p0'] - ann_combine_final3['return_index_p0']
ann_combine_final3['abret_m1'] = ann_combine_final3['return_m1'] - ann_combine_final3['return_index_m1']

# calcualte the variance horizontally, for the abnormal returns from m1 to p1
ann_combine_final3['abret_var_pm1'] = ann_combine_final3[['abret_p1', 'abret_p0', 'abret_m1']].var(axis="columns")

# calculate the variance horizontally, for the abnormal returns from m120 to m21
varlist_benchmark = []
for i in range (21, 120):
    varlist_benchmark.append(f"abret_m{i}")

ann_combine_final3['abret_var_m21m120'] = ann_combine_final3[varlist_benchmark].var(axis="columns")

# Finally, calcuate the abnormal return variance measurement
ann_combine_final3['abretvar_overbenchmark_pm1'] = ann_combine_final3['abret_var_pm1'] / ann_combine_final3['abret_var_m21m120']

# some final cleaning
ann_combine_final3['abretvar_overbenchmark_pm1'] = ann_combine_final3['abretvar_overbenchmark_pm1'].replace(np.inf, np.nan)
ann_combine_final3['abvo_pm1'] = ann_combine_final3['abvo_pm1'].replace(np.inf, np.nan)
ann_combine_final3['abvo_p1p0'] = ann_combine_final3['abvo_p1p0'].replace(np.inf, np.nan)

In [51]:
ann_combine_final3.head()

,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,ann_p1,ann_p2,ann_m1,...,return_m1,return_index_p1,return_index_p0,return_index_m1,abret_p1,abret_p0,abret_m1,abret_var_pm1,abret_var_m21m120,abretvar_overbenchmark_pm1
0,1998.0,001004,1999-05-31,AAR CORP,None,000361105,1999-06-24,1999-06-25,1999-06-28,1999-06-23,...,1.047138,0.999292,0.986566,0.998779,0.031289,0.064881,0.048359,0.000282,0.000874,0.322695
1,1999.0,001004,2000-05-31,AAR CORP,None,000361105,2000-06-28,2000-06-29,2000-06-30,2000-06-27,...,0.968037,0.992342,1.004855,0.994860,0.023616,-0.118063,-0.026824,0.005157,0.001751,2.945161
2,2000.0,001004,2001-05-31,AAR CORP,None,000361105,2001-06-28,2001-06-29,2001-07-02,2001-06-27,...,0.990196,1.000000,1.012804,0.996645,0.110390,0.003698,-0.006449,0.004190,0.001044,4.014264
3,2001.0,001004,2002-05-31,AAR CORP,None,000361105,2002-06-27,2002-06-28,2002-07-01,2002-06-26,...,0.958115,0.998682,1.015093,0.997310,0.086424,0.012229,-0.039194,0.003988,0.000533,7.482761
4,2002.0,001004,2003-05-31,AAR CORP,None,000361105,2003-07-03,2003-07-04,2003-07-07,2003-07-02,...,1.030899,1.000000,0.993095,1.011065,0.000000,-0.038054,0.019834,0.000865,0.001660,0.521360


In [71]:
final_columns = ['FYEAR', 'GVKEY', 'DATADATE', 'CONM', 'SEDOL', 'CUSIP', 'ann_p0', 'CAR1','CAR2','abvo_pm1','abvo_p1p0',
                 'abretvar_overbenchmark_pm1','matched_by','Match','Index']
final_data = ann_combine_final3[final_columns]
final_data = final_data.drop_duplicates()
final_data.head()

,FYEAR,GVKEY,DATADATE,CONM,SEDOL,CUSIP,ann_p0,CAR1,CAR2,abvo_pm1,abvo_p1p0,abretvar_overbenchmark_pm1,matched_by,Match,Index
0,1998.0,001004,1999-05-31,AAR CORP,None,000361105,1999-06-24,0.097733,0.203228,0.833347,0.961662,0.322695,CUSIP,000361105,TOTMKUS
1,1999.0,001004,2000-05-31,AAR CORP,None,000361105,2000-06-28,-0.096217,-0.125370,4.061066,5.823377,2.945161,CUSIP,000361105,TOTMKUS
2,2000.0,001004,2001-05-31,AAR CORP,None,000361105,2001-06-28,0.115909,0.025816,1.254270,1.712412,4.014264,CUSIP,000361105,TOTMKUS
3,2001.0,001004,2002-05-31,AAR CORP,None,000361105,2002-06-27,0.100999,-0.086481,1.206018,1.419308,7.482761,CUSIP,000361105,TOTMKUS
4,2002.0,001004,2003-05-31,AAR CORP,None,000361105,2003-07-03,-0.038054,-0.136474,1.289939,0.866365,0.521360,CUSIP,000361105,TOTMKUS


In [72]:
final_data.to_pickle('final_data.pkl')
final_data.to_csv('final_data.csv')

In [73]:
final_data.to_stata('final_data.dta')

In [74]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154551 entries, 0 to 158708
Data columns (total 15 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   FYEAR                       154551 non-null  float64       
 1   GVKEY                       154551 non-null  object        
 2   DATADATE                    154551 non-null  datetime64[ns]
 3   CONM                        154551 non-null  object        
 4   SEDOL                       113190 non-null  object        
 5   CUSIP                       41361 non-null   object        
 6   ann_p0                      154551 non-null  datetime64[ns]
 7   CAR1                        154353 non-null  float64       
 8   CAR2                        154343 non-null  float64       
 9   abvo_pm1                    151445 non-null  float64       
 10  abvo_p1p0                   150550 non-null  float64       
 11  abretvar_overbenchmark_pm1  150752 non-

In [90]:
# Print some basic information
print(f"There are {len(final_data.GVKEY.unique())} unique firms")
print(" ") # empty line
print(f"{len(final_data.loc[(final_data.SEDOL.isnull()==False) | (final_data.CUSIP.isnull()==False)].GVKEY.unique())} unique firms have either CUSIP or SEDOL number")
print(f"{len(final_data.loc[(final_data.SEDOL.isnull()) & (final_data.CUSIP.isnull())].GVKEY.unique())} unique firms have neither CUSIP nor SEDOL number")
print(" ") # empty line
print(f"{len(final_data[final_data.CUSIP.isnull()==False].GVKEY.unique())} unique firms have the CUSIP number")
print(f"{len(final_data[final_data.SEDOL.isnull()==False].GVKEY.unique())} unique firms have the SEDOL number")
print(" ") # empty line
print(f"There are {sum(final_data.duplicated())} duplicates in the final data.")
# print("The GVKEYs for firms with neither CUSIP nor SEDOL numbers are:")
# for i in final_data.loc[(final_data.SEDOL.isnull()) & (final_data.CUSIP.isnull())].GVKEY.unique():
#     print(i)

There are 23417 unique firms
 
23417 unique firms have either CUSIP or SEDOL number
0 unique firms have neither CUSIP nor SEDOL number
 
6355 unique firms have the CUSIP number
17062 unique firms have the SEDOL number
 
There are 0 duplicates in the final data.
